# Chinese Word Segmentation
## Artificial Intelligence Programming
### Emil Magni (ID: 2021400566)

## 1. Implementing a traditional matching algorithm
### Forward Maximum Matching

In [1]:
# Forward Maximum matching algorithm
# First i made this algorithm that uses the method of forward maximum matching to segment the provided sentence.
# This algorithm requires a file with known words where there is just one word on each line.
# It is to show an example of a simple matching without using machine learning. 

# the forward_matching class is defined with the constructor to create a new matching dictionary based on the file of known words.
class forward_matching(object):
    def __init__(self, dict_path):
        self.dictionary = set() 
        self.maximum = 0
        with open(dict_path, 'r', encoding='utf8') as file:
            for line in file:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                if len(line) > self.maximum:
                    self.maximum = len(line) 

# the match function is used to look up the dictionary and if there is not match it will remove the last word and try again.
    def match(self, text):
        result = []
        size = self.maximum
        text_len = len(text)
        while text_len > 0:
            word = text[0:size]
            while word not in self.dictionary:
                if len(word) == 1:
                  break
                word = word[0:len(word) - 1]
            result.append(word)
            text = text[len(word):]
            text_len = len(text)
        return result

if __name__ == '__main__':
    text = "圆月当空月光如水世界一片朦胧"
    tokenizer = forward_matching('maximum_matching_dictionary.utf8')
    print(tokenizer.match(text))

['圆月', '当空', '月光如水', '世界', '一片', '朦胧']


### Reverse Maximum Matching

In [2]:
# Reverse Maximum matching algorithm
class reverse_matching(object):
    def __init__(self, dict_path):
        self.dictionary = set()
        self.maximum = 0
        with open(dict_path, 'r', encoding='utf8') as file:
            for line in file:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                if len(line) > self.maximum:
                    self.maximum = len(line)

# the reverse_matching is different as it starts from the last character in the input text
    def match(self, text):
        result = []
        index = len(text)
        while index > 0:
            word = None
            for size in range(self.maximum, 0, -1):
                if index - size < 0:
                    continue
                piece = text[(index - size):index]
                if piece in self.dictionary:
                    word = piece
                    result.append(word)
                    index -= size
                    break
            if word is None:
                index -= 1
        return result[::-1]

if __name__ == '__main__':
    text = "我爱吃火锅"
    tokenizer = reverse_matching('maximum_matching_dictionary.utf8')
    print(tokenizer.match(text))

['我', '爱', '吃', '火锅']


# GUI in tkinter for the reverse matching algorithm
The GUI will need an input field where a test string can be entered
When the segment button is pressed then the code will get the input and use it in the code.

In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import showinfo

# creating the window
root = tk.Tk()
root.geometry("300x150")
root.resizable(False, False)
root.title('Word Segmentation')

# First I create a string variable for storing the input text
input_text = tk.StringVar()


# get the text from the input field and use reverse matching before displaying the result in a message box when the segment button is clicked.
def segment_clicked():
    # callback for when the segment button is clicked
   
    text = input_text.get()
    print(text) # printing the input below to see it in jupyter notebook
    rm = reverse_matching('maximum_matching_dictionary.utf8')
    print(tokenizer.match(text)) # printing the result below
    msg = f'You entered the input: {input_text.get()} and the segmented text output is {tokenizer.match(text)}'
    showinfo(
        title='Result',
        message=msg
    )


# input frame
segmentation_frame = ttk.Frame(root)
segmentation_frame.pack(padx=10, pady=10, fill='x', expand=True)


# adding a label describing the input field
input_label = ttk.Label(segmentation_frame, text="Input string:")
input_label.pack(fill='x', expand=True)

# I create the Entry widget for the text input 
input_entry = ttk.Entry(segmentation_frame, textvariable=input_text)
input_entry.pack(fill='x', expand=True)
input_entry.focus()


# segment button
segment_button = ttk.Button(segmentation_frame, text="Segment text", command=segment_clicked)
segment_button.pack(fill='x', expand=True, pady=10)


root.mainloop()

## 2. Implementing a machine learning algorithm based on the Hidden Markov Model
Here I will create a word segmentation with the Hidden Markov Model on the given dataset. I will be using the msr datasets of simplified Chinese corpus for training and testing.

## Loading Data

In [1]:
# Import the necessary libraries
import os
import numpy as np

import re
# Use regular expression to detect Chinese characters
regex = re.compile("[^\u4e00-\u9fff]")

In [2]:
# loading of data
# I store the data in a list with an element for each of the sentences.
train_data = open(os.path.join("icwb2-data/training", "msr_training.utf8"), encoding="utf-8").read().splitlines()
test_data = open(os.path.join("icwb2-data/testing", "msr_test.utf8"), encoding="utf-8").read().splitlines()
test_gold = open(os.path.join("icwb2-data/gold", "msr_test_gold.utf8"), encoding="utf-8").read().splitlines()

In [3]:
# taking a look at the three first sentences of the training data list.
# as we can see the training file has the examples of segmented Chinese words separated by spaces.
train_data[:3]

['“  人们  常  说  生活  是  一  部  教科书  ，  而  血  与  火  的  战争  更  是  不可多得  的  教科书  ，  她  确实  是  名副其实  的  ‘  我  的  大学  ’  。',
 '“  心  静  渐  知  春  似  海  ，  花  深  每  觉  影  生  香  。',
 '“  吃  屎  的  东西  ，  连  一  捆  麦  也  铡  不  动  呀  ？']

In [4]:
# As seen in the testing data the sentences have no segmentation
test_data[:3]

['扬帆远东做与中国合作的先行', '希腊的经济结构较特殊。', '海运业雄踞全球之首，按吨位计占世界总数的１７％。']

In [5]:
test_gold[:3]

['扬帆  远东  做  与  中国  合作  的  先行  ',
 '希腊  的  经济  结构  较  特殊  。',
 '海运  业  雄踞  全球  之  首  ，  按  吨位  计  占  世界  总数  的  １７％  。']

## Data Preprocessing
The training and testing data is preprocessed according to the placement of the characters with 4 labels. The 4 labels will show where the character is placed according to the segmentation.
<ul>
    <li>"S" means it is a single character token standing alone.</li>
    <li>"E" means it is the ending character of a word.</li>
    <li>"M" means it is an internal character in the middle of a word.</li>
    <li>"B" means the character is the beginning of a new word.</li>
</ul>




In [6]:
def preprocess(data):
    hidden_state_lst = []
    data_lst = []
    word_count = {}

# the for loop goes through the sentence and labels the characters by there placement in the segmentation.
    for sentence in data:
        hidden_state = ""
        words = []
        for token in sentence.split():
            token = regex.sub("", token)
            # Each character is getting a label according to the lenght of the word.
            if len(token) == 1:
                hidden_state += "S"
            elif len(token) == 2:
                hidden_state += "BE"
            elif len(token) > 2:
                hidden_state += "B" + (len(token) - 2)*"M" + "E"

        # I remove the spaces
        sentence = sentence.replace(" ", "")
        for word in sentence:
            # regular expression to verify if the word is a Chinese character
            word = regex.sub("", word)
            # creating a dictionary to count the words for use in calculating probabilities.
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
            words.append(word)
        if len(words) > 0:
            data_lst.append(words)
            states = [s for s in hidden_state]
            hidden_state_lst.append(states)

    return data_lst, hidden_state_lst, word_count

# I create dictionaries for the training data and testing data
# the hidden states for the testing is based on the gold dataset with the correct segmentation
# I also create a dictionary for storing of the word count (wc) for both the training and testing data.
train_data, train_hs, train_wc = preprocess(train_data)
test_data, _, test_wc = preprocess(test_data)
_, test_hs, _ = preprocess(test_gold)

In [7]:
# Creating a dictionary for the state count
state_count = {}
states = ["B", "M", "E", "S"]
for state in states: 
    state_count[state] = 0

for i in range(len(train_hs)):
    length = len(train_hs[i])
    if length > 0:
        for j in range(length - 1):
            # Update the state count
            state_count[train_hs[i][j]] += 1
        state_count[train_hs[i][length - 1]] += 1

total_states = sum(state_count.values())  # Getting the total number of states in the sample
start_prob = {}

for state in states:
    # Normalize the state count with the total number of states
    start_prob[state] = state_count[state] / total_states
    
print(start_prob)

{'B': 0.34577982777611344, 'M': 0.09436133454366076, 'E': 0.34577982777611344, 'S': 0.21407900990411233}


In [8]:
# Initializing the transition probabilities
trans_prob = {}
for state in states:
    trans_prob[state] = {}
    for state_i in states:
        trans_prob[state][state_i] = 0

for i in range(len(train_hs)):
    length = len(train_hs[i])
    if length > 0:
        for j in range(length - 1):
            # Update the transition probabilities
            s_from = train_hs[i][j]
            s_to = train_hs[i][j+1]
            trans_prob[s_from][s_to] += 1

for i in states:
    for j in states:
        # Normalize the frequency of the transition with the state counts
        trans_prob[i][j] /= float(state_count[i])

print(trans_prob)

{'B': {'B': 0.0, 'M': 0.14804196105988887, 'E': 0.8519580389401111, 'S': 0.0}, 'M': {'B': 0.0, 'M': 0.4575116593413479, 'E': 0.542488340658652, 'S': 0.0}, 'E': {'B': 0.5755412261794932, 'M': 0.0, 'E': 0.0, 'S': 0.36684089371227374}, 'S': {'B': 0.606061801705827, 'M': 0.0, 'E': 0.0, 'S': 0.37270019136301763}}


In [9]:
# Initialize the emission probabilities
emission_prob = {}

# Get all the vocabulary in the corpus (train and test sets)
vocab = list(set(train_wc.keys()) | set(test_wc.keys()))

# Initialize the emission probabilities
for state in states:
    emission_prob[state] = {}
    for word in vocab:
        emission_prob[state][word] = 1

for i in range(len(train_hs)):
    length = len(train_hs[i])
    for j in range(length):
        # Update the emission probabilities
        obs = train_data[i][j]
        hidden = train_hs[i][j]
        emission_prob[hidden][obs] += 1

for state in states:
    for word in vocab:
        if emission_prob[state][word] == 0:
            continue
        else:
            # Normalize the emission probabilities
            emission_prob[state][word] /= float(state_count[state])

In [10]:
# the Viterbi algorithm below is used to find the most likely sequence of states.
def viterbi_decoding(obs):
    obs = [i for i in obs if i]
    if len(obs) > 0:
        # Initializing a list of dictionary to store the probabilites
        V = [{}]
        for st in states:
            # Append the initial probabilites
            V[0][st] = {"prob": start_prob[st] * emission_prob[st][obs[0]], "prev": None}
        for t in range(1, len(obs)):
            # Append a dictionary to store the probabilities at time/step t
            V.append({})
            for st in states:
                max_tr_prob = V[t - 1][states[0]]["prob"] * trans_prob[states[0]][st]
                prev_st_selected = states[0]
                for prev_st in states[1:]:
                    # Calculate the probabilities of each state
                    tr_prob = V[t - 1][prev_st]["prob"] * trans_prob[prev_st][st]
                    if tr_prob > max_tr_prob:
                        max_tr_prob = tr_prob
                        prev_st_selected = prev_st

                # Get the max probability at time/step t
                max_prob = max_tr_prob * emission_prob[st][obs[t]]
                V[t][st] = {"prob": max_prob, "prev": prev_st_selected}

        path = []
        max_prob = -float("inf")
        best_st = None
        # Get most probable state and its backtrack
        for st, data in V[-1].items():
            if data["prob"] > max_prob:
                max_prob = data["prob"]
                best_st = st
        path.append(best_st)
        previous = best_st

        # Follow the backtrack till the first observation
        for t in range(len(V) - 2, -1, -1):
            path.insert(0, V[t + 1][previous]["prev"])
            previous = V[t + 1][previous]["prev"]

        # Return the path
        return path
    else:
        return []

In [11]:
# the sentences of the testing data is then labelled based on the Viterbi algorithm
test_pred = []
for obs in test_data:
    test_pred.append(viterbi_decoding(obs))

In [12]:
# Based on the labels for the test data I calculate the F1 score to evaluate the model's performance.
# I create a dictionary to store the mapping for each label
map_dict = {
    "B": 0,
    "M": 1,
    "E": 2,
    "S": 3
}

# Flatten the lists into a 1D list
true = list(np.concatenate(test_hs).flat)
pred = list(np.concatenate(test_pred).flat)

k = len(np.unique(true)) # Number of classes
result = np.zeros((k, k)) # Initialize the confusion matrix

for i in range(len(true)):
    # Calculate the confusion matrix
    result[map_dict[true[i]]][map_dict[pred[i]]] += 1

# Precision for each label
precision_B = result[0][0] / (result[0][0] + result[1][0] + result[2][0] + result[3][0])
precision_M = result[1][1] / (result[0][1] + result[1][1] + result[2][1] + result[3][1])
precision_E = result[2][2] / (result[0][2] + result[1][2] + result[2][2] + result[3][2])
precision_S = result[3][3] / (result[0][3] + result[1][3] + result[2][3] + result[3][3])


# Recall for each label
recall_B = result[0][0] / (result[0][0] + result[0][1] + result[0][2] + result[0][3])
recall_M = result[1][1] / (result[1][0] + result[1][1] + result[1][2] + result[1][3])
recall_E = result[2][2] / (result[2][0] + result[2][1] + result[2][2] + result[2][3])
recall_S = result[3][3] / (result[3][0] + result[3][1] + result[3][2] + result[3][3])


# F1 for each label
f1_B = 2 *(precision_B * recall_B)/(precision_B + recall_B)
f1_M = 2 *(precision_M * recall_M)/(precision_M + recall_M)
f1_E = 2 *(precision_E * recall_E)/(precision_E + recall_E)
f1_S = 2 *(precision_S * recall_S)/(precision_S + recall_S)

# Calculating Macro-F1
macro_f1 = (f1_B + f1_M + f1_E + f1_S) / k

print(f"F1 score for state B: {f1_B}")
print(f"F1 score for state M: {f1_M}")
print(f"F1 score for state E: {f1_E}")
print(f"F1 score for state S: {f1_S}")
print(f"Macro-F1 score: {macro_f1}")

F1 score for state B: 0.6738491525681829
F1 score for state M: 0.24104662322574127
F1 score for state E: 0.6812865953626356
F1 score for state S: 0.44521343198634034
Macro-F1 score: 0.5103489507857251


In [13]:
# I apply the word breaking to the testing data
def word_segmentation(test_data, pred):
    segmented = ""
    i = 0 # Counter for the test data index
    j = 0 # Counter for the prediction index
    while i < len(test_data):
        segmented += test_data[i] 
        # Check for Chinese character
        if test_data[i] > u"\u4e00" and test_data[i] < u"\u9fff":
            # Add space after the character if label
            # is either "E" or "S"
            if pred[j] in ["E", "S"]:
                segmented += " "
            j += 1
        i += 1
    return segmented

segmented = []
for i in range(len(test_data)):
    # Converting BMES label to word segmentation
    segmented.append(word_segmentation(test_data[i], test_pred[i]))

In [14]:
# printing some random segmented sentence to see how it works
i = np.random.randint(0, len(test_gold))
print(test_gold[i])
print(test_pred[i])
print(segmented[i])

国际  互联网络  的  发展  给  社会  带来  了  方便  和  巨大  的  商业  价值  ，  但  同时  也  产生  了  一些  副作用  。
['B', 'E', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'B', 'E', 'B', 'E', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E']
国际 互联 网络 的 发展 给 社会 带来 了方 便和 巨大 的 商业 价值 但 同时 也 产生 了 一些副 作 用


In [15]:
# I save the file with segmentation as my_prediction.txt
with open("my_prediction.txt", "w", encoding="utf-8") as fout:
    for item in segmented:
        fout.write("%s\n" % item)

## tkinter GUI for HMM Machine Learning algorithm
I am using tkinter to create a simple ui interface for the word segmentation.

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import showinfo

# creating the window
root = tk.Tk()
root.geometry("300x150")
root.resizable(False, False)
root.title('Word Segmentation with HMM')

# First I create a string variable for storing the input text
input_text = tk.StringVar()


# get the text from the input field and use reverse matching before displaying the result in a message box when the segment button is clicked.
def segment_clicked():
    # callback for when the segment button is clicked

    text = input_text.get()
    print(text) # printing the input below to see it in jupyter notebook
    #rm = reverse_matching('maximum_matching_dictionary.utf8')
    print(word_segmentation(text, pred)) # printing the result below
    msg = f'You entered the input: {input_text.get()} and the segmented text output is {word_segmentation(text, pred)}'
    showinfo(
        title='Result',
        message=msg
    )


# input frame
segmentation_frame = ttk.Frame(root)
segmentation_frame.pack(padx=10, pady=10, fill='x', expand=True)


# adding a label describing the input field
input_label = ttk.Label(segmentation_frame, text="Input string:")
input_label.pack(fill='x', expand=True)

# I create the Entry widget for the text input 
input_entry = ttk.Entry(segmentation_frame, textvariable=input_text)
input_entry.pack(fill='x', expand=True)
input_entry.focus()


# segment button
segment_button = ttk.Button(segmentation_frame, text="Segment text", command=segment_clicked)
segment_button.pack(fill='x', expand=True, pady=10)


root.mainloop()

圆月当空月光如水世界一片朦胧
圆月 当空 月光 如水 世界 一片 朦胧 
我们不得不同心协力对付这些问题
我们 不得 不同 心协 力对 付 这些 问题 
我要出国学习英语但将  会  回来
我要 出国 学习 英语 但将   会   回来 
我要出国学习英语但将会回来
我要 出国 学习 英语 但将 会 回来 
